In [5]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Assuming df is your DataFrame with columns 'URL' and 'URL_ID'
df = pd.read_excel("links.xlsx")  # Load your DataFrame
rows = df.shape[0]  # Get the number of rows (i.e., the number of URLs)

# Create a directory to save the text files, if it doesn't exist
os.makedirs('textfiles', exist_ok=True)

# Loop through the rows in the DataFrame
for i in range(rows):  # 'rows' is now an integer
    url = df.URL[i]  # Get the URL from the 'URL' column
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Check for different class names for titles
        title_element = soup.find('h1', class_='entry-title') or soup.find('h1', class_='tdb-title-text')

        if title_element:
            title = title_element.text.strip()

            # Find the article content
            article = soup.find('div', class_='td-post-content tagdiv-type') or soup.find('div', class_='tdb-block-inner td-fix-index')

            if article:
                paragraphs = article.find_all('p')
                all_text = '\n'.join([p.get_text() for p in paragraphs])

                # Clean and encode the article text
                all_text = all_text.replace('\n', ' ').replace('#', ' ')
                article_encoded = all_text.encode('utf-8')

                # Save the article content into a text file with URL_ID as filename
                file_path = os.path.join('textfiles', f'{df.URL_ID[i]}.txt')
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(title + '\n')
                    f.write(article_encoded.decode('utf-8'))

        else:
            print(f"Title not found for URL {url}")

    else:
        print(f"Failed to retrieve URL {url}")


FileNotFoundError: [Errno 2] No such file or directory: 'links.xlsx'